This project is based on the available kaggle competition - Detection of toxic comment.  
link: https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge

Let's import necessary packages first.

In [48]:
from string import digits
import string
import sys, os, re, csv, codecs, numpy as np, pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from gensim.models import KeyedVectors
from sklearn.preprocessing import Imputer
from nltk.tokenize import wordpunct_tokenize
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier, LogisticRegressionCV
from sklearn.svm import LinearSVC, NuSVC
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

In [38]:
#dataset=pd.read_csv('datasets/tow_classes_tweet.csv', encoding='utf-8', usecols=['sentiment', 'txt'])
dataset1=pd.read_csv('C:/Users/aas3n17/Desktop/Organized/datasets/Ar_tweet_cleaned_final.csv', encoding='utf-8', usecols=['sentiment', 'txt'])
dataset2=pd.read_csv('C:/Users/aas3n17/Desktop/Organized/datasets/qrci.csv', encoding='utf-8', usecols=['sentiment', 'txt'])
dataset3=pd.read_csv('C:/Users/aas3n17/Desktop/Organized/datasets/astd.csv', encoding='utf-8', usecols=['sentiment', 'txt'])

In [39]:
dataset1.sentiment.value_counts()

1    993
0    958
Name: sentiment, dtype: int64

In [40]:
dataset2.sentiment.value_counts()

1    377
0    377
Name: sentiment, dtype: int64

In [41]:
dataset3.sentiment.value_counts()

0    812
1    777
Name: sentiment, dtype: int64

In [50]:
dataset1=clean(dataset1)
dataset2=clean(dataset2)
dataset3=clean(dataset3)

In [54]:
dataset3.txt.head(100)

0                     عن اي مشروب غازي كوﻻ حافظمشفاهم  
1     عاده لا اعلن عن مواقفي السياسيهلكن كله الا الد...
2     انتا فرحه جات ل عندي بعد دنيا من التعب السعاده...
3     البلاوي تصنيف الاخوان تنظيما ارهابيا قرار سياس...
4      فكره المقاله حلوه واسلوبك في توصيل الفكره حلو...
5     اهالي عزبهالنخل يفضحون السيسي حملته قالت لنا ا...
6     الساده الافاضل بجريده المصري اليوم عن الخبر ال...
7     الحمد له ان حقن دماء المصرين واصلح بينهم وعقبا...
8     يا مرسي يا عظيم تسريب سيسي كشف انه كان عملاقا ...
9     العبيد في اعلام العار لم ينطق بعد تصريح ايهاب ...
10    اسرار نشاه السيسي داخل حاره اليهود بالقاهره ال...
11    لبيك الهم لبيك لبيك لا شريك لك لبيك ان الحمد و...
12    نحن امه اذا وضعت لمستخدمي الانترنت فيها كلمه ف...
13    الهم امطر علي قبر محمد حسين جمعه وعلي قبور جمي...
14    حفيد السلطان عبدالحميد مرشح لبرلمان التركي الق...
15    من اجمل ذكريات الطفوله انك تنام في اي مكان في ...
16    المصري بيقول حزام ناسف و الشروق بيقول سياره مس...
17    ع منطق حقير ولا يتبعه سوي الجبناء هذه نفس 

In [ ]:
train, test = train_test_split(dataset, test_size=0.1)
train.head()
#prints all the categories

In [7]:
# Little preprocessing required
sentences_train = train["txt"].fillna("_na_").values
classes = ["sentiment"]
y = train[classes].values
sentences_test = test["txt"].fillna("_na_").values

In [22]:
# Embedding parameter set
embed_size = 100 # how big is each word vector
max_features = 10000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 50 # max number of words in a comment to use

In [23]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(sentences_train))
tokens_train = tokenizer.texts_to_sequences(sentences_train)
tokens_test = tokenizer.texts_to_sequences(sentences_test)
X_train = pad_sequences(tokens_train, maxlen=maxlen)
X_test = pad_sequences(tokens_test, maxlen=maxlen)

# Create Keras Model

In [24]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size)(inp)
x = LSTM(4, return_sequences=True, dropout=0.2, recurrent_dropout=0.1)(x)
x = Conv1D(16,4,activation='relu')(x)
x = Flatten()(x)
x = Dense(100, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer=optimizers.rmsprop(lr = 0.001,decay = 1e-06), metrics=['accuracy'])
filepath="Weights/weights-improvement.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 50)                0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 50, 100)           1000000   
_________________________________________________________________
lstm_3 (LSTM)                (None, 50, 4)             1680      
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 47, 16)            272       
_________________________________________________________________
flatten_3 (Flatten)          (None, 752)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 100)               75300     
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
__________

Fit the defined model onto the data:

In [25]:
model.fit(X_train, y, batch_size=32, epochs=5,callbacks=callbacks_list, verbose=1, validation_split=0.2)

Train on 1404 samples, validate on 351 samples
Epoch 1/5
1404/1404 [==============================] - 3s 2ms/step - loss: 0.6678 - acc: 0.5848 - val_loss: 0.6238 - val_acc: 0.5897

Epoch 00001: val_acc improved from -inf to 0.58974, saving model to Weights/weights-improvement.hdf5
Epoch 2/5
1404/1404 [==============================] - 2s 1ms/step - loss: 0.4524 - acc: 0.8027 - val_loss: 0.4049 - val_acc: 0.8120

Epoch 00002: val_acc improved from 0.58974 to 0.81197, saving model to Weights/weights-improvement.hdf5
Epoch 3/5
1404/1404 [==============================] - 2s 1ms/step - loss: 0.2513 - acc: 0.9010 - val_loss: 0.3310 - val_acc: 0.8547

Epoch 00003: val_acc improved from 0.81197 to 0.85470, saving model to Weights/weights-improvement.hdf5
Epoch 4/5
1404/1404 [==============================] - 2s 1ms/step - loss: 0.1458 - acc: 0.9459 - val_loss: 0.3378 - val_acc: 0.8490

Epoch 00004: val_acc did not improve from 0.85470
Epoch 5/5
1404/1404 [==============================] - 2s 

# Select from saved weights as per choice and predict response

In [26]:
from keras.models import load_model
saved_model = load_model('Weights/weights-improvement.hdf5')
y_pred = saved_model.predict(X_test)

In [27]:
y_pred=y_pred.round()
y_test=test.sentiment

In [21]:
        f1_score = metrics.f1_score(y_test, y_pred)

        # macro accuracy (macro average)
        macc = metrics.f1_score(y_test, y_pred, pos_label=None, average='macro')

        # precision and recall
        recall = metrics.recall_score(y_test, y_pred)
        precision = metrics.precision_score(y_test, y_pred)

        results = [macc, f1_score, precision, recall]
        print(results)

[0.8342494714587738, 0.813953488372093, 0.8235294117647058, 0.8045977011494253]


In [36]:
    def clean(df):
        path = "Arabic_stop_word.txt"
        stop_words = []
        with codecs.open(path, "r", encoding="utf-8", errors="ignore") as myfile:
            stop_words = myfile.readlines()
        stop_words = [word.strip() for word in stop_words]
        arabic_punctuations = '''؛<>_()*،&^%][ـ،/:"؟ـ`÷×.,'{}~¦+|!”…“–_'''
        arabic_numbers = '''۰۱۲۳٤٥٦٧۸۹'''
        english_punctuations = string.punctuation
        punctuations_list = arabic_punctuations + english_punctuations

        arabic_diacritics = re.compile("""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
        for comments in df:
            # removearabic_diacritics
            df['txt'] = df['txt'].map(lambda x: re.sub(arabic_diacritics,'',str(x)))

            #def normalize_arabic(text)
            df['txt'] = df['txt'].map(lambda x: re.sub("[إأآا]", "ا",str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub("ؤ", "ء",str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub("ى", "ي",str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub("ئ", "ء",str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub("ة", "ه",str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub("گ", "ك",str(x)))

            

            #def remove_punctuations(text):
            df['txt'] = df['txt'].map(lambda x: re.sub("["+punctuations_list+"]",'',str(x)))

            # remove_repeating_char(text):
            df['txt'] = df['txt'].map(lambda x: re.sub(r'(.)\1+', r'\1', str(x)))
            # remove '\\n'
            df['txt'] = df['txt'].map(lambda x: re.sub('\\n',' ',str(x)))

            # remove any text starting with User... 
            df['txt'] = df['txt'].map(lambda x: re.sub("\[\[User.*",'',str(x)))

            # remove IP addresses or user IDs
            df['txt'] = df['txt'].map(lambda x: re.sub("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}",'',str(x)))

            # lower uppercase letters
            #df['txt'] = df['txt'].map(lambda x: str(x).lower())

            #remove http links in the text
            df['txt'] = df['txt'].map(lambda x: re.sub("(http://.*?\s)|(http://.*)",'',str(x)))

            #remove all punctuation
            df['txt'] = df['txt'].map(lambda x: re.sub("_", '',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub("«", ' ',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub("»", ' ',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub("“", ' ',str(x))) 
            df['txt'] = df['txt'].map(lambda x: re.sub("”", ' ',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub("😞", ' ',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub("😔", ' ',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub("😂", ' ',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub("🌹", ' ',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub("✨", ' ',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub("👉", ' ',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub("👈", ' ',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub("☹", ' ',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub("👇", ' ',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub("ک", 'ك',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub("—", ' ',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub("ლ", ' ',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub("╹", ' ',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub("◡", ' ',str(x)))
            
            
            df['txt'] = df['txt'].map(lambda x: re.sub("♥", ' ',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub("♡", ' ',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub("¥", ' ',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub("؟", ' ',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub("!", ' ',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub("#", ' ',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub("$", ' ',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub("%", ' ',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub("&", ' ',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub("✖", ' ',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub('ة', 'ه',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub(' ئ', 'ء',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub('ؤ', 'ء',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub('ے', 'ك',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub('ڪ', 'ك',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub('أ', 'ا',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub('إ', 'ا',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub('آ', 'ا',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub('\\n',' ',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub('\\r',' ',str(x)))
            df['txt']= df['txt'].map(lambda x: re.sub("["+string.punctuation+"]",'',str(x)))
            df['txt']= df['txt'].map(lambda x: re.sub("["+arabic_numbers+"]",'',str(x)))
            df['txt']= df['txt'].map(lambda x: re.sub("["+punctuations_list+"]",'',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub("["+digits+"]",'',str(x)))
            #df['txt'] = df['txt'].map(lambda x: re.sub(r'[^\x600-\x6ff]','',str(x)))
            df['txt'] = df['txt'].map(lambda x: re.sub(r'[a-zA-Z?]','',str(x))) 
            df['txt'].apply(lambda x: [item for item in x if item not in stop_words])
        return df